In [1]:
import pandas as pd
!pip3 install ijson
import ijson

train_json_data = open("data-set/train.json", 'r')
jsonparsed_train_data = ijson.parse(train_json_data)

temp ={'message':[],'values':[]}
for prefix, event, value in jsonparsed_train_data:
#     print("prefix=> {0} \n event => {1} \n value => {2} \n\n".format(prefix,event,value))
    if event == 'string':
        temp['message'].append((prefix))
        temp['values'].append((value))
print(len(temp['message']))
train_data = pd.DataFrame(temp)

test_csv_data = open("/data-set/test.csv",'r')
test_data = pd.read_csv(test_csv_data,names=["ID","message"])
test_data = test_data.drop('ID',axis='columns')
print(train_data.shape)
print(test_data.shape)

     |████████████████████████████████| 126 kB 2.3 MB/s eta 0:00:01
4152659
(4152659, 2)
(595301, 1)


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
#Vital functions we will use for preprocessing data
import re

def _preprocess(data):
    for col in data.columns:
        if col == "message":
            data[col] = data[col].apply(_clean_message)
        else:
            data[col] = data[col].apply(to_str)

    data = data.fillna("EMPTY")
    
def _clean_message(line):
    """Remove all none alphabetical and non numerical characters from message strings."""
    return "".join(
        re.findall("[a-zA-Z0-9:-_.]+", line)
    )  # Leaving only a-z in there as numbers add to anomalousness quite a bit

def to_str(x):
    """Convert all non-str lists to string lists for Word2Vec."""
    ret = " ".join([str(y) for y in x]) if isinstance(x, list) else str(x)
    return ret



In [3]:
import pandas as pd

print(train_data.info())
# print(train_data.head())
# print(train_data.tail())

print(test_data.info())
# print(test_data.head())
# print(test_data.tail())

# concating train and test to preprocess them together we will split them before training
data = pd.concat([train_data,test_data])

print(data.info())
print(data.head())
#apophis@21

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4152659 entries, 0 to 4152658
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   message  object
 1   values   object
dtypes: object(2)
memory usage: 63.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595301 entries, 0 to 595300
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   message  595301 non-null  object
dtypes: object(1)
memory usage: 4.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4747960 entries, 0 to 595300
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   message  object
 1   values   object
dtypes: object(2)
memory usage: 108.7+ MB
None
                                             message  values
0   1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...  normal
1   1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...  normal
2   1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...  normal
3  

In [4]:
_preprocess(data)
data.head()

,message,values
0,11178385702005.06.03R02M1N0C:J12U112005060315....,normal
1,11178385702005.06.03R02M1N0C:J12U112005060315....,normal
2,11178385702005.06.03R02M1N0C:J12U112005060315....,normal
3,11178385702005.06.03R02M1N0C:J12U112005060315....,normal
4,11178385702005.06.03R02M1N0C:J12U112005060315....,normal


In [5]:
data.tail()

,message,values
595296,11187682342005.06.14R20M0NAC:J17U112005061409....,nan
595297,11183709472005.06.09R15M0NAC:J02U012005060919....,nan
595298,11185402422005.06.11R30M0N9C:J16U012005061118....,nan
595299,11185389472005.06.11R30M0N9C:J16U012005061118....,nan
595300,11181658052005.06.07R23M1N5C:J16U112005060710....,nan


In [6]:
new_train = data.iloc[:4152659]
new_test = data.iloc[4152660:]

# new_test.drop("values", axis = "columns")


In [7]:
from sklearn.model_selection import train_test_split
X = new_train['message']
y = new_train['values'].map({'normal':1,'abnormal':0}).astype(int)
test_size = 0.1
random_state = 1234

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state, stratify=y)

In [8]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
with tpu_strategy.scope():
    class GensimWord2VecVectorizer(BaseEstimator, TransformerMixin):
        """
        Word vectors are averaged across to create the document-level vectors/features.
        gensim's own gensim.sklearn_api.W2VTransformer doesn't support out of vocabulary words,
        hence we roll out our own.
        All the parameters are gensim.models.Word2Vec's parameters.
        https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
        """

        def __init__(self, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None,
                 sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5,
                 ns_exponent=0.75, cbow_mean=1, hashfxn=hash, iter=5, null_word=0,
                 trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False,
                 callbacks=(), max_final_vocab=None):
            self.size = size
            self.alpha = alpha
            self.window = window
            self.min_count = min_count
            self.max_vocab_size = max_vocab_size
            self.sample = sample
            self.seed = seed
            self.workers = workers
            self.min_alpha = min_alpha
            self.sg = sg
            self.hs = hs
            self.negative = negative
            self.ns_exponent = ns_exponent
            self.cbow_mean = cbow_mean
            self.hashfxn = hashfxn
            self.iter = iter
            self.null_word = null_word
            self.trim_rule = trim_rule
            self.sorted_vocab = sorted_vocab
            self.batch_words = batch_words
            self.compute_loss = compute_loss
            self.callbacks = callbacks
            self.max_final_vocab = max_final_vocab

        def fit(self, X, y=None):
            self.model_ = Word2Vec(
            sentences=X, corpus_file=None,
            vector_size=self.size, alpha=self.alpha, window=self.window, min_count=self.min_count,
            max_vocab_size=self.max_vocab_size, sample=self.sample, seed=self.seed,
            workers=self.workers, min_alpha=self.min_alpha, sg=self.sg, hs=self.hs,
            negative=self.negative, ns_exponent=self.ns_exponent, cbow_mean=self.cbow_mean,
            hashfxn=self.hashfxn, null_word=self.null_word,
            trim_rule=self.trim_rule, sorted_vocab=self.sorted_vocab, batch_words=self.batch_words,
            compute_loss=self.compute_loss, callbacks=self.callbacks,
            max_final_vocab=self.max_final_vocab)
            return self

        def transform(self, X):
            X_embeddings = np.array([self._get_embedding(words) for words in X])
            return X_embeddings

        def _get_embedding(self, words):
            valid_words = [word for word in words if word in self.model_.wv.index_to_key]
            if valid_words:
                embedding = np.zeros((len(valid_words), self.size), dtype=np.float32)
                for idx, word in enumerate(valid_words):
                    embedding[idx] = self.model_.wv[word]

                return np.mean(embedding, axis=0)
            else:
                return np.zeros(self.size)


    gensim_word2vec_tr = GensimWord2VecVectorizer(size=50, min_count=3, sg=1, alpha=0.025, iter=10)
    xgb = XGBClassifier(learning_rate=0.01, n_estimators=100, n_jobs=-1)
    w2v_xgb = Pipeline([
        ('w2v', gensim_word2vec_tr), 
        ('xgb', xgb)
    ])
    w2v_xgb

2022-12-16 16:14:14.093564: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-12-16 16:14:14.093699: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 16:14:19.308563: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-16 16:14:19.310930: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-12-16 16:14:19.310959: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-16 16:14:19.310987: I tensorflow/stream_executor/cuda/cud

In [9]:
import time

start = time.time()

w2v_xgb.fit(X_train, y_train)
elapse = time.time() - start
print('elapsed: ', elapse)
w2v_xgb



/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:06:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
elapsed:  5390.725734949112


Pipeline(steps=[('w2v',
                 GensimWord2VecVectorizer(iter=10, min_count=3, sg=1, size=50)),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='', learning_rate=0.01,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=-1, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_train_pred = w2v_xgb.predict(X_train)
print('Training set accuracy %s' % accuracy_score(y_train, y_train_pred))
confusion_matrix(y_train, y_train_pred)

Training set accuracy 0.9999480921594277


array([[  62531,     192],
       [      2, 3674668]])

In [11]:
y_test_pred = w2v_xgb.predict(X_test)
print('Test set accuracy %s' % accuracy_score(y_test, y_test_pred))
confusion_matrix(y_test, y_test_pred)

Test set accuracy 0.9999518380989535


array([[  6950,     19],
       [     1, 408296]])

In [12]:
new_test.drop('values',axis='columns')
test = new_test['message']
print(test)

1         11243363012005.08.17R13M1N8C:J12U012005081720....
2         11185531752005.06.11R30M0N9C:J16U012005061122....
3         11185360332005.06.11R30M0N9C:J16U012005061117....
4         11179925662005.06.05R30M1N6C:J03U112005060510....
5         11185389652005.06.11R30M0N9C:J16U012005061118....
                                ...                        
595296    11187682342005.06.14R20M0NAC:J17U112005061409....
595297    11183709472005.06.09R15M0NAC:J02U012005060919....
595298    11185402422005.06.11R30M0N9C:J16U012005061118....
595299    11185389472005.06.11R30M0N9C:J16U012005061118....
595300    11181658052005.06.07R23M1N5C:J16U112005060710....
Name: message, Length: 595300, dtype: object


In [13]:
final_pred = w2v_xgb.predict(test)

In [14]:
my_pred = pd.DataFrame(final_pred,columns=['Label'])
my_pred = my_pred['Label'].map({0:'abnormal',1:'normal'})
print(my_pred)

0           normal
1         abnormal
2         abnormal
3           normal
4         abnormal
            ...   
595295      normal
595296      normal
595297    abnormal
595298    abnormal
595299      normal
Name: Label, Length: 595300, dtype: object


In [15]:
final = pd.DataFrame(my_pred,columns=['Label'])
# again getting test data for id
test_csv_data = open("/kaggle/input/convolve-epoch1/test.csv",'r')
test_data = pd.read_csv(test_csv_data,names=["ID","message"])
test_data.drop(index=test_data.index[0], axis=0, inplace=True)
print(test_data)
print(final)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


            ID                                            message
1            0   1124336301 2005.08.17 R13-M1-N8-C:J12-U01 200...
2            1   1118553175 2005.06.11 R30-M0-N9-C:J16-U01 200...
3            2   1118536033 2005.06.11 R30-M0-N9-C:J16-U01 200...
4            3   1117992566 2005.06.05 R30-M1-N6-C:J03-U11 200...
5            4   1118538965 2005.06.11 R30-M0-N9-C:J16-U01 200...
...        ...                                                ...
595296  595295   1118768234 2005.06.14 R20-M0-NA-C:J17-U11 200...
595297  595296   1118370947 2005.06.09 R15-M0-NA-C:J02-U01 200...
595298  595297   1118540242 2005.06.11 R30-M0-N9-C:J16-U01 200...
595299  595298   1118538947 2005.06.11 R30-M0-N9-C:J16-U01 200...
595300  595299   1118165805 2005.06.07 R23-M1-N5-C:J16-U11 200...

[595300 rows x 2 columns]
           Label
0         normal
1       abnormal
2       abnormal
3         normal
4       abnormal
...          ...
595295    normal
595296    normal
595297  abnormal
595298  abn

In [16]:
test_data.reset_index(drop=True, inplace=True)
final.reset_index(drop=True,inplace=True)
sub = pd.concat([test_data.ID,final.Label],axis=1)
sub.set_index('ID',inplace=True)
print(sub)

           Label
ID              
0         normal
1       abnormal
2       abnormal
3         normal
4       abnormal
...          ...
595295    normal
595296    normal
595297  abnormal
595298  abnormal
595299    normal

[595300 rows x 1 columns]


In [17]:
sub.to_csv(f"submission_data_sniffers.csv")